In [68]:
import requests
import json
import pandas as pd
import time
from fake_useragent import UserAgent
import pymssql

db_settings = {
    "host": "127.0.0.1",
    "user": "sa",
    "password": "zxc123",
    "database": "ncu_database",
    "charset": "utf8"
}

In [71]:
try:  
    all_url = []
    all_stock =[]
    conn = pymssql.connect(**db_settings)
    with conn.cursor() as cursor:
        command = "select * from [dbo].[stock_list]"
        cursor.execute(command)
        result = cursor.fetchall()
        for year in range(2022, 2023):
            for month in range(1, 3):
                for r in result:
                    try:
                        stock = r[0]
                        date = str(year)+f"{month:02d}"+"01"
                        if r[2] == "上市":
                            address = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock}"
                        elif r[2] == "上櫃":
                            address = f"http://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d={date}&stkno={stock}"
                        all_url.append(address)
                        all_stock.append(r[0])
                    except Exception as ex:
                        continue                                     
        conn.close()           
except Exception as ex:
    print(ex)

In [80]:
len(all_url)

3500

In [81]:
try:  
    user_agent = UserAgent()
    conn = pymssql.connect(**db_settings)
    with conn.cursor() as cursor:
        insert_command = "INSERT INTO [dbo].[stock_data](c, d, v, z, o, h, l, tz, u, tv) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        for index in range(3049, len(all_url)):
            try:
                response = requests.get(url=all_url[index], headers={ 'user-agent': user_agent.random })
                data = response.text  # 這是json格式的資料
                a_json = json.loads(data)  # 轉成dict
                if("tpex" in all_url[index]):
                    data_str = "aaData"
                else:
                    data_str = "data"
                    
                for data in a_json[data_str]:
                    date = data[0].split("/")
                    data[0] = str(1911 + int(date[0])) + date[1] + date[2] + " 14:00:00"
                    data[1] = data[1].replace(",", "")
                    data[2] = data[2].replace(",", "")
                    data[3] = data[3].replace(",", "")
                    data[4] = data[4].replace(",", "")
                    data[5] = data[5].replace(",", "")
                    data[6] = data[6].replace(",", "")
                    data[7] = data[7].replace(",", "")
                    data[8] = data[8].replace(",", "")
                    cursor.execute(insert_command, (all_stock[index], data[0], data[1], data[2], data[3], data[4], data[5], data[6], data[7], data[8]))
                    conn.commit()
                time.sleep(10)
                print(time.strftime("%H:%M:%S", time.localtime()) + " " + str(index))
            except Exception as ex:
                time.sleep(10)
                print(time.strftime("%H:%M:%S", time.localtime()) + " error " + str(index) + " " + data)
                continue
        conn.close()
except Exception as e:
#    print(e)
    error_class = e.__class__.__name__ #取得錯誤類型
    detail = e.args[0] #取得詳細內容
    cl, exc, tb = sys.exc_info() #取得Call Stack
    lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
    fileName = lastCallStack[0] #取得發生的檔案名稱
    lineNum = lastCallStack[1] #取得發生的行號
    funcName = lastCallStack[2] #取得發生的函數名稱
    errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
    print(errMsg)

14:21:03 3049
14:21:14 3050
14:21:24 3051
14:21:34 3052
14:21:44 3053
14:21:54 3054
14:22:04 3055
14:22:14 3056
14:22:24 3057
14:22:35 3058
14:22:45 3059
14:22:55 3060
14:23:05 3061
14:23:15 3062
14:23:25 3063
14:23:35 3064
14:23:46 3065
14:23:56 3066
14:24:06 3067
14:24:16 3068
14:24:27 3069
14:24:37 3070
14:24:47 3071
14:24:57 3072
14:25:07 3073
14:25:17 3074
14:25:27 3075
14:25:37 3076
14:25:48 3077
14:25:58 3078
14:26:08 3079
14:26:18 3080
14:26:28 3081
14:26:38 3082
14:26:48 3083
14:26:59 3084
14:27:09 3085
14:27:19 3086
14:27:29 3087
14:27:39 3088
14:27:49 3089
14:27:59 3090
14:28:09 3091
14:28:20 3092
14:28:30 3093
14:28:40 3094
14:28:50 3095
14:29:00 3096
14:29:10 3097
14:29:20 3098
14:29:30 3099
14:29:41 3100
14:29:51 3101
14:30:01 3102
14:30:11 3103
14:30:29 3104
14:30:39 3105
14:30:49 3106
14:31:00 3107
14:31:10 3108
14:31:20 3109
14:31:30 3110
14:31:40 3111
14:31:50 3112
14:32:00 3113
14:32:11 3114
14:32:24 3115
14:32:34 3116
14:32:44 3117
14:32:55 3118
14:33:05 3119
14:33: